In [1]:
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
from pathlib import Path

In [2]:
# from typing import Literal
# from datasets import Dataset, DatasetDict, load_dataset

# class YambdaDataset:
#     INTERACTIONS = frozenset([
#         "likes", "listens", "multi_event", "dislikes", "unlikes", "undislikes"
#     ])

#     def __init__(
#         self,
#         dataset_type: Literal["flat", "sequential"] = "flat",
#         dataset_size: Literal["50m", "500m", "5b"] = "50m"
#     ):
#         assert dataset_type in {"flat", "sequential"}
#         assert dataset_size in {"50m", "500m", "5b"}
#         self.dataset_type = dataset_type
#         self.dataset_size = dataset_size

#     def interaction(self, event_type: Literal[
#         "likes", "listens", "multi_event", "dislikes", "unlikes", "undislikes"
#     ]) -> Dataset:
#         assert event_type in YambdaDataset.INTERACTIONS
#         return self._download(f"{self.dataset_type}/{self.dataset_size}", event_type)

#     def audio_embeddings(self) -> Dataset:
#         return self._download("", "embeddings")

#     def album_item_mapping(self) -> Dataset:
#         return self._download("", "album_item_mapping")

#     def artist_item_mapping(self) -> Dataset:
#         return self._download("", "artist_item_mapping")

#     @staticmethod
#     def _download(data_dir: str, file: str) -> Dataset:
#         data = load_dataset("yandex/yambda", data_dir=data_dir, data_files=f"{file}.parquet")
#         # Returns DatasetDict; extracting the only split
#         assert isinstance(data, DatasetDict)
#         return data["train"]


In [3]:
# dataset = YambdaDataset("sequential", "50m")
# data = dataset.interaction("multi_event")

In [4]:
# df = data.to_pandas()
# df.head()

In [5]:
# user = df.iloc[0]

In [6]:
# user[['item_id', 'timestamp', 'event_type', 'played_ratio_pct', 'track_length_seconds']]

In [7]:
# a = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# print(len(a[2: 7]))
# print(7-2)

In [8]:
# # ----------------- НАСТРОЙКИ -----------------
# INPUT_PATH = 'multi_event.parquet'   # твой файл
# OUTPUT_PATH = 'yambda_sessions_threshold_900.parquet'  # куда сохраним
# THRESHOLD = 900          # 75 минут
# BATCH_SIZE = 5           #
# DROPOUT_BATCH_RATE = 0.6
# CHUNK_SIZE = 2_000      # размер чанка — подбери под свою RAM (10k нормально)
# # -------------------------------------------

# def split_sessions_for_user(row, threshold):
#     timestamps = np.array(row['timestamp'], dtype=np.int64)
#     if len(timestamps) < 2:
#         return [row]  # одна сессия

#     diffs = np.diff(timestamps)
#     split_idx = np.where(diffs > threshold)[0] + 1
#     starts = np.concatenate([[0], split_idx])
#     ends = np.concatenate([split_idx, [len(timestamps)]])

#     sessions = []
#     for i in range(len(starts)):
#         start, end = starts[i], ends[i]
#         length = end - start

#         # Filter the length of the session
#         if length < BATCH_SIZE:
#             continue
#         elif length < 2*BATCH_SIZE:
#           if np.random.uniform(0, 1) < DROPOUT_BATCH_RATE:
#             continue

#         sessions.append({
#             'uid': row['uid'],
#             'session_idx': i,
#             'session_length': length,
#             'item_ids': row['item_id'][start:end],
#             'played_ratio_pct': row['played_ratio_pct'][start:end] if row['played_ratio_pct'] is not None else None,
#             'event_type': row['event_type'][start:end]
#         })
#     return sessions

# # ----------------- ОСНОВНОЙ ЦИКЛ -----------------
# print("Загружаем весь датасет...")
# df = pd.read_parquet(INPUT_PATH, columns=['uid', 'timestamp', 'item_id', 'played_ratio_pct', 'event_type'])

# all_sessions = []
# for _, row in df.iterrows():
#     row_dict = row.to_dict()
#     sessions = split_sessions_for_user(row_dict, THRESHOLD)
#     all_sessions.extend(sessions)

# # Создаём DF и сохраняем
# sessions_df = pd.DataFrame(all_sessions)
# sessions_df.to_parquet(OUTPUT_PATH, engine='pyarrow', index=False, compression='snappy')

# print(f"\nГотово! Файл: {OUTPUT_PATH}")
# print(f"Количество сессий: {len(sessions_df)}")
# print(sessions_df.head())


In [9]:
# sessions_df.head()

In [10]:
# data = pd.read_parquet('sessions_threshold_900.parquet')

In [11]:
# embed_ids = pd.read_parquet('embeddings.parquet', columns=['item_id'])

In [12]:
# embed_ids = embed_ids['item_id'].unique()


In [13]:
# data_ids = pd.Series(data['item_ids'].explode()).unique()

In [14]:
# missing = np.intersect1d(data_ids, embed_ids)
# print(len(missing))

In [6]:
import polars as pl
from tqdm import tqdm
import numpy as np
import time  # для оценки времени

# Settings
SESSIONS_PATH = 'sessions_threshold_900.parquet'
EMBEDDINGS_PATH = 'embeddings.parquet'
OUTPUT_PATH = 'sessions_cleaned.parquet'

start_time = time.time()  # для общей оценки времени

print("Loading embeddings...")
valid_items = pl.read_parquet(EMBEDDINGS_PATH, columns=['item_id'])
valid_item_ids = valid_items['item_id'].unique().to_list()  # уже уникальные, no extra unique
print(f"Initial valid item_ids from embeddings: {len(valid_item_ids):,}")

print("Loading sessions...")
sessions_pl = pl.read_parquet(SESSIONS_PATH)
print(f"Original sessions: {len(sessions_pl):,}")

# Добавляем пересечение с уникальными item_ids из сессий (ускорение is_in)
print("Computing intersection with session item_ids...")
session_item_ids = sessions_pl.select(pl.col('item_ids').explode()).unique().to_series().to_list()
valid_item_ids = np.intersect1d(session_item_ids, valid_item_ids, assume_unique=True).tolist()
valid_item_ids_set = set(valid_item_ids)  # для быстрого lookup
print(f"Optimized valid item_ids (intersection): {len(valid_item_ids):,}")

# Explode to long format (lazy — no RAM spike yet)
print("Exploding and filtering...")
long_pl = sessions_pl.explode(['item_ids', 'played_ratio_pct', 'event_type'])  # timestamps убрал

# Filter with progress simulation (Polars fast, but add tqdm for estimate)
# Polars не поддерживает tqdm напрямую, но мы можем симулировать на батчах если нужно
# Для скорости — делаем в один pass
long_pl = long_pl.filter(pl.col('item_ids').is_in(valid_item_ids))

print(f"Filtered rows: {long_pl.height:,}")
elapsed = time.time() - start_time
print(f"Elapsed so far: {elapsed:.2f} sec")

# Group back
print("Grouping back...")
grouped = long_pl.group_by(['uid', 'session_idx']).agg([
    pl.col('item_ids').alias('item_ids'),
    pl.col('played_ratio_pct').alias('played_ratio_pct'),
    pl.col('event_type').alias('event_type'),
    pl.len().alias('session_length')
])

# To Pandas or save directly
cleaned_df = grouped.to_pandas()

# Statistics (FIXED HERE: col.sum() instead of sum(col))
total_tracks_before = sessions_pl.select(pl.col('session_length').sum()).item()
total_tracks_after = cleaned_df['session_length'].sum()
print(f"Removed tracks: {total_tracks_before - total_tracks_after:,}")

# Save
cleaned_df.to_parquet(OUTPUT_PATH)
total_elapsed = time.time() - start_time
print(f"Total time: {total_elapsed:.2f} sec")
print("Done!")

Loading embeddings...
Initial valid item_ids from embeddings: 7,721,749
Loading sessions...
Original sessions: 1,663,411
Computing intersection with session item_ids...
Optimized valid item_ids (intersection): 807,703
Exploding and filtering...
Filtered rows: 41,346,820
Elapsed so far: 8.06 sec
Grouping back...
Removed tracks: 1,396,665
Total time: 30.39 sec
Done!


In [7]:
print(pd.Series(cleaned_df['item_ids'].explode()).nunique())


807703
